# 4)

In [9]:
import pandas as pd

df = pd.read_csv('msr_paraphrase_corpus.txt', delimiter='\t', usecols=[0, 1, 2, 3, 4])
df.head()

,Quality,ID1,ID2,String1,String2
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi..."
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an..."
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ..."
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...


In [10]:
incorrect_matches = df[df['Quality'] == 0]
incorrect_matches.head()

,Quality,ID1,ID2,String1,String2
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ..."
6,0,264589,264502,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...
8,0,3114205,3114194,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...
10,0,142746,142671,"Gyorgy Heizler, head of the local disaster uni...","The head of the local disaster unit, Gyorgy He..."


In [13]:
print("Length of incorrect matches: ", len(incorrect_matches))

Length of incorrect matches:  1285


In [12]:
# Define a list of words quantifiers such as negation
quantifiers = ['not', 'no', 'never', 'none', 'nobody', 'nothing', 'neither', 'nowhere', 'hardly', 'scarcely', 'barely']

# Join the list into a single string pattern
pattern = '|'.join(quantifiers)

mask = incorrect_matches['String1'].str.contains(pattern, case=False, na=False) | incorrect_matches['String2'].str.contains(pattern, case=False, na=False)

# Filter the DataFrame
result_df = incorrect_matches[mask]
result_df.head()

,Quality,ID1,ID2,String1,String2
8,0,3114205,3114194,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...
19,0,1713015,1712982,A BMI of 25 or above is considered overweight;...,A BMI between 18.5 and 24.9 is considered norm...
28,0,1783137,1782659,No dates have been set for the civil or the cr...,No dates have been set for the criminal or civ...
56,0,2506257,2506206,The women then had follow-up examinations afte...,The women had follow-up examinations in 1974-7...
60,0,2277501,2277502,"American Masters: Arthur Miller, Elia Kazan an...",Note the subheading of this terrible parable i...


In [14]:
print("Length of incorrect matches with quantifiers: ", len(result_df))

Length of incorrect matches with quantifiers:  408


In [18]:
percentage = round(len(result_df) / len(incorrect_matches) * 100, 2)
print(f"Percentage of incorrect matches with quantifiers: {percentage}%")

Percentage of incorrect matches with quantifiers: 31.75%


In [3]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
from nltk.metrics import edit_distance
import numpy as np

def leacock_chodorow_similarity(word1, word2):
    # Get the synsets for each word
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)

    # Initialize an empty list to store the path lengths
    path_lengths = []

    # Calculate the path length for each pair of synsets
    for synset1 in synsets1:
        for synset2 in synsets2:
            path_length = synset1.shortest_path_distance(synset2)
            if path_length is not None:
                path_lengths.append(path_length)

    # Calculate the shortest path length
    if len(path_lengths) > 0:
        shortest_path_length = min(path_lengths)
    else:
        shortest_path_length = np.inf

    # Calculate the length of the longest common subsequence
    lcs_length = edit_distance(word1, word2)

    # Calculate the similarity score
    similarity_score = -np.log(lcs_length / (2 * shortest_path_length))

    return similarity_score


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\luigi\AppData\Roaming\nltk_data...


Similarity score between 'dog' and 'cat': 0.9808292530117262
